In [5]:
# Installing Python packages
%pip install pyarrow
%pip install pandas
%pip install xarray

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Importing necessary libraries
import pyarrow as pa
import pandas as pd
import xarray as xr
import os
import requests
import json
from io import BytesIO

In [ ]:
# Set your Beacon Blue Cloud Token
TOKEN = ''

In [8]:
# Fetching available columns from the Beacon API
responseinfo = requests.get("https://beacon-wb2-eutrophication.maris.nl/api/query/available-columns", headers = {"Authorization" : f"Bearer {TOKEN}"}) 
params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").

In [48]:
# Search function to find columns based on a search term
# This function searches for columns in the params list that match the search term
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]
    
    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")

search_columns("L05") #Enter your search term here

Matching columns:
COMMON_CHLOROPHYLL_L05
COMMON_OXYGEN_L05
COMMON_PHOSPHATE_L05
COMMON_SILICATE_L05
COMMON_SALINITY_L05
COMMON_TEMPERATURE_L05
COMMON_OXYGEN_SATURATION_L05
COMMON_NITRATE_L05
COMMON_AMMONIUM_L05


In [49]:
# You JSON query request
query = {
    "query_parameters": [
            {
                "column_name": "SOURCE_BDI",
            },
            {
                "column_name": "SOURCE_BDI_DATASET_ID",
            },
            {
                "column_name": "COMMON_CHLOROPHYLL_L05",
            },
            {
                "column_name": "COMMON_CHLOROPHYLL_L22",
            },
            {
                "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME",
            },
            {
                "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME_QC",
            },
            {
                "column_name": "COMMON_CHLOROPHYLL_PER_VOLUME_P01",
            },
            {
                "column_name": "COMMON_TIME",
            },
            {
                "column_name": "COMMON_DEPTH",
                "alias": "DEPTH"
            },
            {
                "column_name": "COMMON_LONGITUDE",
                "alias": "LONGITUDE"
            },
            {
                "column_name": "COMMON_LATITUDE",
                "alias": "LATITUDE"
            }
        ],
        "filters": [
            {
                "for_query_parameter": "COMMON_TIME",
                "min": f"2015-01-01T00:00:00",
                "max": f"2016-01-01T00:00:00",
            },
            {
              "for_query_parameter": "DEPTH",
              "min": 0,
              "max": 5
            },
            {
                "is_not_null": {
                    "for_query_parameter": "COMMON_CHLOROPHYLL_PER_VOLUME",
                }
            },
        ],
        "output": {
            "format": "parquet"
        }
    }


In [50]:
response = requests.post("https://beacon-wb2-eutrophication.maris.nl/api/query", json.dumps(query), headers = {
    "Authorization" : f"Bearer {TOKEN}",
    "Content-type": "application/json"
})

if response.status_code == 204:
    print("No data has been found for your query, please update your input fields above and run the notebook again.")
elif response.status_code != 200:
    # Print error message if the request was not successful
    print(f"Error: {response.status_code}")
    print(response.text)

In [ ]:
# Wrap the response content in a BytesIO object
response_content = BytesIO(response.content)
# Read the Parquet file into a Pandas DataFrame
df = pd.read_parquet(response_content, engine='pyarrow')
# This is a temporary fix as the merged instance works with seconds since 1970 which isn't supported by parquet (but will be addressed by beacon in the future)
df['COMMON_TIME'] = pd.to_datetime(df['COMMON_TIME'], unit='s')
df

,SOURCE_BDI,SOURCE_BDI_DATASET_ID,COMMON_CHLOROPHYLL_L05,COMMON_CHLOROPHYLL_L22,COMMON_CHLOROPHYLL_PER_VOLUME,COMMON_CHLOROPHYLL_PER_VOLUME_QC,COMMON_CHLOROPHYLL_PER_VOLUME_P01,COMMON_TIME,DEPTH,LONGITUDE,LATITUDE
0,BEACON_EMODNET_CHEMISTRY,48640100001_PCh_Surf,SDN:L05::999,,0.7400,2,SDN:P01::CHLTVOLU,2015-01-12 12:57:28,1.000000,5.906421,43.087318
1,BEACON_EMODNET_CHEMISTRY,48640100001_PCh_Surf,SDN:L05::999,,0.3200,2,SDN:P01::CHLTVOLU,2015-01-27 11:11:07,1.000000,5.906421,43.087318
2,BEACON_EMODNET_CHEMISTRY,48640100001_PCh_Surf,SDN:L05::999,,0.4300,2,SDN:P01::CHLTVOLU,2015-02-10 12:01:34,1.000000,5.906421,43.087318
3,BEACON_EMODNET_CHEMISTRY,48640100001_PCh_Surf,SDN:L05::999,,0.4700,2,SDN:P01::CHLTVOLU,2015-02-23 11:12:11,1.000000,5.906421,43.087318
4,BEACON_EMODNET_CHEMISTRY,48640100001_PCh_Surf,SDN:L05::999,,0.1800,2,SDN:P01::CHLTVOLU,2015-03-09 12:02:38,1.000000,5.906421,43.087318
...,...,...,...,...,...,...,...,...,...,...,...
1187593,BEACON_WOD,wod_021756605O,None,SDN:L22::TOOL0673,1.1952,1,SDN:P01::CHLTVOLU,2015-12-21 23:39:55,3.890044,-70.486107,39.671982
1187594,BEACON_WOD,wod_021756605O,None,SDN:L22::TOOL0673,1.4040,1,SDN:P01::CHLTVOLU,2015-12-21 23:39:55,4.148054,-70.486107,39.671982
1187595,BEACON_WOD,wod_021756605O,None,SDN:L22::TOOL0673,1.0008,1,SDN:P01::CHLTVOLU,2015-12-21 23:39:55,4.396141,-70.486107,39.671982
1187596,BEACON_WOD,wod_021756605O,None,SDN:L22::TOOL0673,1.2672,1,SDN:P01::CHLTVOLU,2015-12-21 23:39:55,4.644228,-70.486107,39.671982


In [52]:
# Create output directory if it doesn't exist
if not os.path.exists("./beacon_output"):
    os.makedirs("beacon_output")
    
# Save the response content to a file
with open("beacon_output/merged_subset.parquet", "wb") as f:
    f.write(response.content)